In [1]:
# Import libraries and dependencies
import numpy as np
import pandas as pd
import os
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import alpaca_trade_api as tradeapi
%matplotlib inline

In [2]:
# Set the random seed for resutls reproducibility (for testing purposes only)
np.random.seed(42)

In [3]:
# Load .env enviroment variables
from dotenv import load_dotenv
load_dotenv()

# Set Alpaca API key and secret
alpaca_api_key = 'PKJQI8S3KRE4YRDLRM2U'
alpaca_secret_key = 'BnlDGVWNTHKzkLB7tzChxSe62w0NSoFVbaiEiMou'

# Create the Alpaca API object
api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

In [4]:
 # Subset your tickers, then pull returns data:
# Set the ticker
ticker = ["SPY", "AGG"]

# Set timeframe to '1D'
timeframe = "1D"

# Set start and end datetimes of 1 year, between now and 365 days ago.
start_date = pd.Timestamp('2019-01-01', tz='America/New_York').isoformat()
end_date = pd.Timestamp('2019-12-31', tz='America/New_York').isoformat()

# Get 1 year's worth of historical data for SPY and AGG
df = api.get_barset(
    ticker,
    timeframe,
    limit=None,
    start=start_date,
    end=end_date,
    after=None,
    until=None,
).df

# Drop unnecessary columns
df = df.drop(
    columns=['open', 'high', 'low', 'volume'],
    level=1
)

In [5]:
 # Display sample data
df.head()

AGG     SPY
                             close   close
2019-01-02 00:00:00-05:00  106.585  249.89
2019-01-03 00:00:00-05:00  107.020  244.46
2019-01-04 00:00:00-05:00  106.695  252.41
2019-01-07 00:00:00-05:00  106.470  254.40
2019-01-08 00:00:00-05:00  106.420  256.65

In [6]:
 # Calculate the daily roi for the stocks
daily_returns = df.pct_change()
# Display sample data
daily_returns.dropna()

AGG       SPY
                              close     close
2019-01-03 00:00:00-05:00  0.004081 -0.021730
2019-01-04 00:00:00-05:00 -0.003037  0.032521
2019-01-07 00:00:00-05:00 -0.002109  0.007884
2019-01-08 00:00:00-05:00 -0.000470  0.008844
2019-01-09 00:00:00-05:00  0.000564  0.004130
...                             ...       ...
2019-12-24 00:00:00-05:00  0.001292  0.000125
2019-12-26 00:00:00-05:00  0.000934  0.005136
2019-12-27 00:00:00-05:00  0.001245 -0.000155
2019-12-30 00:00:00-05:00  0.000089 -0.005451
2019-12-31 00:00:00-05:00 -0.002308  0.002554

[251 rows x 2 columns]

In [7]:
 # Compute daily volatility
volatility = daily_returns.std() * np.sqrt(252)

In [9]:
 # Save the last day's closing price
daily_returns.tail()

AGG       SPY
                              close     close
2019-12-24 00:00:00-05:00  0.001292  0.000125
2019-12-26 00:00:00-05:00  0.000934  0.005136
2019-12-27 00:00:00-05:00  0.001245 -0.000155
2019-12-30 00:00:00-05:00  0.000089 -0.005451
2019-12-31 00:00:00-05:00 -0.002308  0.002554

In [25]:
# Setup the Monte Carlo Parameters
number_simulations = 500
number_records = 252 * 30
monte_carlo = pd.DataFrame(daily_returns)

In [27]:
monte_carlo.dropna()

AGG       SPY
                              close     close
2019-01-03 00:00:00-05:00  0.004081 -0.021730
2019-01-04 00:00:00-05:00 -0.003037  0.032521
2019-01-07 00:00:00-05:00 -0.002109  0.007884
2019-01-08 00:00:00-05:00 -0.000470  0.008844
2019-01-09 00:00:00-05:00  0.000564  0.004130
...                             ...       ...
2019-12-24 00:00:00-05:00  0.001292  0.000125
2019-12-26 00:00:00-05:00  0.000934  0.005136
2019-12-27 00:00:00-05:00  0.001245 -0.000155
2019-12-30 00:00:00-05:00  0.000089 -0.005451
2019-12-31 00:00:00-05:00 -0.002308  0.002554

[251 rows x 2 columns]

In [ ]:
plt.figure()
plot_title = f'Monte Carlo: {n+1} Simulations of 30 Year Return
portfolio 